<a href="https://colab.research.google.com/github/rohanpagadala/Smart-Loan-Recovery-System-with-Machine-Learning/blob/main/Smart_Loan_Recovery_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv("/content/loan_recovery.csv")
df.describe()

,Age,Monthly_Income,Num_Dependents,Loan_Amount,Loan_Tenure,Interest_Rate,Collateral_Value,Outstanding_Loan_Amount,Monthly_EMI,Num_Missed_Payments,Days_Past_Due,Collection_Attempts
count,500.000000,500.000000,500.000000,5.000000e+02,500.00000,500.000000,5.000000e+02,5.000000e+02,500.000000,500.000000,500.000000,500.000000
mean,43.116000,134829.920000,1.476000,1.024907e+06,46.10400,11.192820,6.032240e+05,5.627260e+05,15861.536020,1.912000,70.678000,3.000000
std,12.733217,68969.356746,1.145447,5.907556e+05,18.23706,3.775209,7.457131e+05,4.723581e+05,18709.231315,2.110252,60.211038,2.807805
min,21.000000,15207.000000,0.000000,5.413800e+04,12.00000,5.020000,0.000000e+00,1.571283e+04,261.880000,0.000000,0.000000,0.000000
25%,32.000000,76343.250000,0.000000,4.629848e+05,36.00000,7.907500,0.000000e+00,1.822072e+05,4039.097500,1.000000,4.000000,1.000000
50%,44.000000,134929.500000,1.000000,9.971240e+05,48.00000,10.915000,2.327684e+05,4.133240e+05,9330.170000,2.000000,66.500000,2.000000
75%,53.000000,193086.250000,3.000000,1.557952e+06,60.00000,14.577500,1.111106e+06,8.324787e+05,20439.485000,3.000000,122.250000,4.000000
max,64.000000,249746.000000,3.000000,1.995325e+06,72.00000,17.970000,2.744395e+06,1.932396e+06,127849.230000,12.000000,180.000000,10.000000


In [4]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

features = ['Age', 'Monthly_Income', 'Loan_Amount', 'Loan_Tenure', 'Interest_Rate',
            'Collateral_Value', 'Outstanding_Loan_Amount', 'Monthly_EMI',
            'Num_Missed_Payments', 'Days_Past_Due']

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[features])

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df['Borrower_Segment'] = kmeans.fit_predict(df_scaled)

In [5]:
df['Segment_Name'] = df['Borrower_Segment'].map({
    0: 'Moderate Income, High Loan Burden',
    1: 'High Income, Low Default Risk',
    2: 'Moderate Income, Medium Risk',
    3: 'High Loan, Higher Default Risk'
})

In [6]:
df['High_Risk_Flag'] = df['Segment_Name'].apply(
    lambda x: 1 if x in ['High Loan, Higher Default Risk', 'Moderate Income, High Loan Burden'] else 0
)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = df[features]
y = df['High_Risk_Flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
risk_scores = rf_model.predict_proba(X_test)[:, 1]

In [8]:
df_test = X_test.copy()
df_test['Risk_Score'] = risk_scores
df_test['Predicted_High_Risk'] = (df_test['Risk_Score'] > 0.5).astype(int)

df_test = df_test.merge(
    df[['Borrower_ID', 'Segment_Name', 'Recovery_Status', 'Collection_Method', 'Collection_Attempts', 'Legal_Action_Taken']],
    left_index=True, right_index=True
)

In [9]:
def assign_recovery_strategy(risk_score):
    if risk_score > 0.75:
        return "Immediate legal notices & aggressive recovery attempts"
    elif 0.50 <= risk_score <= 0.75:
        return "Settlement offers & repayment plans"
    else:
        return "Automated reminders & monitoring"

df_test['Recovery_Strategy'] = df_test['Risk_Score'].apply(assign_recovery_strategy)

In [11]:
df_test[['Borrower_ID', 'Risk_Score', 'Predicted_High_Risk',
         'Segment_Name', 'Recovery_Status', 'Recovery_Strategy']].head(10)

,Borrower_ID,Risk_Score,Predicted_High_Risk,Segment_Name,Recovery_Status,Recovery_Strategy
436,BRW_437,0.42,0,"High Loan, Higher Default Risk",Partially Recovered,Automated reminders & monitoring
448,BRW_449,0.64,1,"Moderate Income, High Loan Burden",Fully Recovered,Settlement offers & repayment plans
88,BRW_89,0.76,1,"High Loan, Higher Default Risk",Fully Recovered,Immediate legal notices & aggressive recovery ...
145,BRW_146,0.32,0,"High Income, Low Default Risk",Fully Recovered,Automated reminders & monitoring
344,BRW_345,0.09,0,"High Income, Low Default Risk",Fully Recovered,Automated reminders & monitoring
239,BRW_240,0.97,1,"Moderate Income, High Loan Burden",Partially Recovered,Immediate legal notices & aggressive recovery ...
279,BRW_280,0.12,0,"High Income, Low Default Risk",Partially Recovered,Automated reminders & monitoring
445,BRW_446,0.91,1,"High Loan, Higher Default Risk",Fully Recovered,Immediate legal notices & aggressive recovery ...
320,BRW_321,1.00,1,"Moderate Income, High Loan Burden",Partially Recovered,Immediate legal notices & aggressive recovery ...
121,BRW_122,0.63,1,"Moderate Income, High Loan Burden",Fully Recovered,Settlement offers & repayment plans


In [12]:
import plotly.express as px

fig = px.scatter(df, x='Monthly_Income', y='Loan_Amount',
                 color='Segment_Name', size='Loan_Amount',
                 title='Borrower Segments by Monthly Income and Loan Amount',
                 labels={'Monthly_Income': 'Income ($)', 'Loan_Amount': 'Loan ($)'},
                 color_discrete_sequence=px.colors.qualitative.Vivid)

fig.show()